In [2]:
%pip install ngutils

Note: you may need to restart the kernel to use updated packages.


In [153]:
import io, os
import pandas as pd
import numpy as np
import datetime

from tqdm import tqdm
from ngutils import Tc

import concurrent.futures

import requests
from sklearn.utils import gen_batches

api_key = '6655b49c72afdc5d55edbf58bb4a9b6d'

tc = Tc()
tc.log('start')

start: 2022-11-06 18:40 mem:9.371GB                                                                                                     


In [4]:
# %%time
# df = pd.read_pickle('../data/Full_16_09_22.pickle')

CPU times: user 24.7 s, sys: 7.18 s, total: 31.9 s
Wall time: 31.9 s


In [5]:
%%time
df = pd.read_pickle('../data/Full_16_09_22-cat.pickle')

CPU times: user 10.4 s, sys: 3.42 s, total: 13.9 s
Wall time: 13.9 s


In [23]:
%%time
# анализ наличия дублей заявок
df[df.duplicated(subset=['Номер заявки'], keep=False)].sort_values('Номер заявки')

# Было выявлено 174 дубля

CPU times: user 4.74 s, sys: 84.4 ms, total: 4.82 s
Wall time: 4.8 s


,Корневой ИД заявки,ИД версии заявки,Номер заявки,Уникальный номер обращения ГУ (mos.ru),Дата создания заявки в формате Timezone,Дата начала действия версии заявки в формате Timezone,Наименование источника поступления заявки,Код источника поступления заявки,Имя создателя заявки,Признак инцидента,...,Дата закрытия,Желаемое время с,Желаемое время до,Дата отзыва/оценки,Отзыв,Оценка качества выполнения работ,Код оценки качества выполнения ра,Наименование категории платности,Код категории платности,Признак оплаты картой
1401,73413460,73417854,00001402/21,NaN,2021-01-01 10:32:00.823530+03:00,2021-01-01 12:16:29.483998+03:00,телефон,phone,frontline,Нет,...,2021-01-01 12:16:29.483998+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
1152234,73413460,73417854,00001402/21,NaN,2021-01-01 10:32:00.823530+03:00,2021-01-01 12:16:29.483998+03:00,телефон,phone,frontline,Нет,...,2021-01-01 12:16:29.483998+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
1152235,73418979,73438385,00002693/21,NaN,2021-01-01 12:43:24.546609+03:00,2021-01-01 19:53:36.113079+03:00,телефон,phone,frontline,Нет,...,2021-01-01 19:53:36.113079+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
2692,73418979,73438385,00002693/21,NaN,2021-01-01 12:43:24.546609+03:00,2021-01-01 19:53:36.113079+03:00,телефон,phone,frontline,Нет,...,2021-01-01 19:53:36.113079+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
3394,73422028,73493222,00003395/21,NaN,2021-01-01 13:48:35.566341+03:00,2021-01-03 10:31:30.282955+03:00,телефон,phone,frontline,Нет,...,2021-01-03 10:31:30.282955+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538620,76030186,76047704,00539016/21,NaN,2021-01-31 15:09:30.019257+03:00,2021-01-31 19:45:25.050600+03:00,телефон,phone,frontline,Нет,...,2021-01-31 19:45:25.050600+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
1152406,76034619,76035077,00539965/21,NaN,2021-01-31 16:11:06.318965+03:00,2021-01-31 16:18:11.989116+03:00,Электронный дом,elec_house,ElecH,Нет,...,2021-01-31 16:18:11.989116+03:00,2021-02-01 16:39:00+03:00,2021-02-01 17:39:00+03:00,2021-04-16 01:38:43.991846+03:00,NaN,Неудовлетворительно,awful,бесплатная,free,Нет
539569,76034619,76035077,00539965/21,NaN,2021-01-31 16:11:06.318965+03:00,2021-01-31 16:18:11.989116+03:00,Электронный дом,elec_house,ElecH,Нет,...,2021-01-31 16:18:11.989116+03:00,2021-02-01 16:39:00+03:00,2021-02-01 17:39:00+03:00,2021-04-16 01:38:43.991846+03:00,NaN,Неудовлетворительно,awful,бесплатная,free,Нет
540342,76038412,76174932,00540738/21,NaN,2021-01-31 17:07:45.046024+03:00,2021-02-02 10:35:48.660509+03:00,телефон,phone,frontline,Нет,...,2021-01-31 17:15:44.930874+03:00,NaT,NaT,2021-04-16 01:38:43.991846+03:00,Работы не выполнены,Плохо,bad,бесплатная,free,Нет


In [35]:
api_key = api_key[::-1]

In [35]:
url = 'https://apidata.mos.ru/v1/datasets/60562/count'
params = {'api_key': api_key}
n_rows = int(requests.get(url, params=params).text)
print(f"Количество записей в Адресном реестре объектов недвижимости города Москвы: {n_rows}")

Количество записей в Адресном реестре объектов недвижимости города Москвы: 481614


In [ ]:
url = 'https://apidata.mos.ru/v1/datasets/60562/rows'

In [ ]:
def load_60562(batch):
    params = {'api_key': api_key, '$skip': batch.start, '$top': min(500, batch.stop-batch.start)}
    data = ['SIMPLE_ADDRESS', 'UNOM', 'ADM_AREA', 'DISTRICT', 'geoData']
    r = requests.post(url, params=params, json=data)
    with open(f"../data/apidata_mos_ru_60562/{batch.start:06}-{batch.stop-1:06}.txt", 'wb') as f:
        f.write(r.content)

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    future_to_file = [executor.submit(load_60562, batch) for batch in gen_batches(n_rows, 500)]
    for future in tqdm(concurrent.futures.as_completed(future_to_file), total=n_rows//500):
        pass

In [ ]:
url = 'https://apidata.mos.ru/v1/datasets/60562/rows'
params = {'api_key': api_key,
          '$filter': 'Cells/UNOM eq 5783', '$top':2}
r = requests.get(url, params=params)
print(r.status_code, r.reason)
print(f"Пример выборки по конкретному UNOM{r.text}")

In [60]:
%%time

# Анализ загруженных UNOM и формирование общей таблицы. Выявление пропущенных UNOM

import concurrent.futures
import time

filelist = os.listdir('../data/apidata_mos_ru_60562')
list_df = []
err_fl = []
if os.path.isfile('../data/UNOM_geoData.pickle'):
    set_BATCH = set(pd.read_pickle('../data/UNOM_geoData.pickle')['BATCH'].to_list())
else:
    set_BATCH = set()

def parse_60562(filename):
    try:
        df = pd.read_json(f"../data/apidata_mos_ru_60562/{filename}")
        list_df.append(df.Cells.apply(pd.Series)[['UNOM', 'ADM_AREA', 'DISTRICT', 'SIMPLE_ADDRESS', 'geoData']].assign(BATCH=filename[:-4]).set_index('UNOM'))
    except Exception as exc:
        err_fl.append(int(filename[:6]))
        print('%r generated an exception: %s' % (filename, exc))
        err_fl.append(int(filename[:6]))

with concurrent.futures.ThreadPoolExecutor(max_workers=23) as executor:
    future_to_json = {executor.submit(parse_60562, filename): filename for filename in filelist if filename[:-4] not in set_BATCH}
    for future in tqdm(concurrent.futures.as_completed(future_to_json)):
        pass

time.sleep(.5)
print(f"Обработано {len(list_df)} файлов, ошибок: {len(err_fl)}")

if os.path.isfile('../data/UNOM_geoData.pickle'):
    df = pd.read_pickle('../data/UNOM_geoData.pickle')
    df = pd.concat([df]+list_df)
else:
    df = pd.concat(list_df)
df.to_pickle('../data/UNOM_geoData.pickle')

In [ ]:
%%time
# Дозагрузка пропущенных UNOM

import concurrent.futures
import requests
from tqdm import tqdm

api_key = 'd6b9a4bb85fbde55d5cdfa27c94b5566'
url = 'https://apidata.mos.ru/v1/datasets/60562/rows'

def load_60562(start):
    params = {'api_key': api_key, '$skip': start, '$top': 500}
    data = ['SIMPLE_ADDRESS', 'UNOM', 'ADM_AREA', 'DISTRICT', 'geoData']
    r = requests.post(url, params=params, json=data)
    with open(f"../data/apidata_mos_ru_60562/{start:06}-{start+499:06}.txt", 'wb') as f:
        f.write(r.content)
        print(f"../data/apidata_mos_ru_60562/{start:06}-{start+499:06}.txt")

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_file = [executor.submit(load_60562, i) for i in err_fl]
    try:
        for future in tqdm(concurrent.futures.as_completed(future_to_file)):
            pass
    except Exception as e:
        print(e)
    
# После дозагрузки необходимо повторить Анализ загруженных UNOM (предыдущий шаг)

In [36]:
# Дефекты, заявки по которым могут быть закрыты за 10 минут
# В соответствии с приложением 2
df_control_10m_pril2 = pd.DataFrame(
    columns = ['Описание', 'Корневой идентификатор дефекта'],
    data = [
        ['Оповещение от 112', 2303],
        ['Проникновение в шахту лифта', 2245],
        ['Заблокирована входная дверь подъезда, оборудованная домофоном, кодовым замком (на вход/выход)', 1903],
        ['Сообщить, заменить код домофона', 2396],
        ['Требуется обеспечение водой (отсутствует холодная вода более 4 часов)', 1922],
        ['Застревание пассажира в лифте (в данный момент)', 1771],
        ['Управление освещением подъезда', 2096],
        ['Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала)', 7907],
        ['Подача документов о поверке ИПУ воды в электронном виде', 7906]
    ]
)
    
df_control_10m_pril2

,Описание,Корневой идентификатор дефекта
0,Оповещение от 112,2303
1,Проникновение в шахту лифта,2245
2,"Заблокирована входная дверь подъезда, оборудов...",1903
3,"Сообщить, заменить код домофона",2396
4,Требуется обеспечение водой (отсутствует холод...,1922
5,Застревание пассажира в лифте (в данный момент),1771
6,Управление освещением подъезда,2096
7,"Ввод в эксплуатацию ИПУ воды (замена, демонтаж...",7907
8,Подача документов о поверке ИПУ воды в электро...,7906


In [218]:
df_pril3 = pd.read_csv('../data/pril3.csv')
df_pril3

,Идентификатор,Корневой идентификатор,Наименование,Категория,"Повторное срок, часов",Повторное локация
0,39046,2077,"Аварийный провал грунта, асфальта",Благоустройство,24,адрес
1,39045,1853,Благоустройство дворовой территории после пров...,Благоустройство,72,адрес
2,40442,1960,Благоустройство Другое,Благоустройство,-,-
3,39043,2226,Восстановление бордюрного камня (дворовая терр...,Благоустройство,72,адрес
4,39042,2258,"Грязь, мусор на контейнерной площадке (дворова...",Благоустройство,12,адрес
...,...,...,...,...,...,...
299,38895,38577,"Установка, замена, ремонт розетки/выключателя ...",Электрика,72,адрес+кв.
300,38708,2150,"Частично отсутствует освещение, ток в электрос...",Электрика,24,адрес+кв.
301,40678,1831,"Шум, гул в помещении электрощитовой, трансформ...",Электрика,48,адрес
302,39840,2191,Электрика Другое,Электрика,-,-


## Шаблон Аномалий 1. 'Заявки, закрытые без выполнения. Повторные'
- Заявки в статусах «закрытые», «закрытые через МАРМ», с типом дефектов кроме «Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала)», «Подача документов о поверке ИПУ воды в электронном виде» с результативностью кроме «Выполнено» с любым результатом закрытия, кроме «Аварийное/плановое отключение», поступившие от заявителя по одному и тому же типу дефекта в течение определенного промежутка времени с момента закрытия первичной заявки согласно критериям Приложения 3.

In [67]:
## Методика расчета аномалии

In [ ]:
# Смотрим критерии аномальности по отдельности

In [45]:
sum(df['Наименование дефекта'] == 'Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала)'), \
sum(df['Наименование дефекта'] == 'Подача документов о поверке ИПУ воды в электронном виде'), \
len(df)

(6743, 21621, 6603578)

In [46]:
sum(df['Результативность'] != 'Выполнено'), sum(df['Результативность'] == 'Выполнено')

(2931391, 3672187)

In [54]:
df['Наименование статуса заявки'].value_counts()

Закрыта               6602735
Закрыта через МАРМ        829
В работе                   12
Новая                       2
Name: Наименование статуса заявки, dtype: int64

In [62]:
sum(df['Наименование статуса заявки'].isin(['Закрыта', 'Закрыта через МАРМ']))

6603564

In [65]:
sum(df['Вид выполненных работ'] == 'Аварийное/плановое отключение')

189823

In [66]:
df['Вид выполненных работ'].value_counts()[:10]

Оказана консультация с выходом специалиста на место                  768894
Оказана консультация по телефону                                     667539
Замена лампы                                                         474866
В ходе осмотра проблема не подтвердилась                             368209
Перепущен стояк                                                      255390
Засор устранен                                                       192664
Аварийное/плановое отключение                                        189823
Перенос заявки на более поздний срок по инициативе заявителя         180129
Множественное обращение по данному дефекту (от разных заявителей)    176518
Включен автомат                                                      147272
Name: Вид выполненных работ, dtype: int64

In [81]:
# И все вместе - это 'Заявки, закрытые без выполнения'
df_bez_vypolnenia = df[
    df['Наименование статуса заявки'].isin(['Закрыта', 'Закрыта через МАРМ'])
    &(~df['Наименование дефекта'].isin([
        'Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала)', 
        'Подача документов о поверке ИПУ воды в электронном виде']))
    &(df['Результативность']!='Выполнено')
    &(df['Вид выполненных работ']!='Аварийное/плановое отключение')
]

In [82]:
# Смотрим среди них повторные заявки в соответствии с критериями приложения 3

In [ ]:
# Для каждой заявки считаем, были ли повторные заявки по адресу и квартире

In [219]:
# Выберем категории дефектов, где считаем аномалией повторное обращение по адресу и квартире
df_pril3_sroki = df_pril3[['Повторное локация', 'Наименование', 'Повторное срок, часов']].set_index('Наименование')
df_pril3_sroki['Повторное локация'].value_counts()

-                95
адрес+кв.        86
адрес+подъезд    69
адрес            48
Адрес             6
Name: Повторное локация, dtype: int64

In [220]:
def_adress_kv = df_pril3_sroki[df_pril3_sroki['Повторное локация']=='адрес+кв.'].index.to_list()
# Первые 10 записей
def_adress_kv[:10]

['Вызов специалиста для оказания консультации по вентиляции (вопросы замены, переноса, приобретения оборудования)',
 'Неисправность системы вентиляции в квартире',
 'Проверка, прочистка дымохода (квартира с газовой колонкой, камином)',
 'Вызов специалиста для оказания консультации по водосчетчикам (вопросы диагностики, замены, переноса, приобретения оборудования)',
 'Грызуны (крысы, мыши) в квартире',
 'Заказать, восстановить магнитный ключ для домофона',
 'Неисправна трубка домофона в квартире',
 'Вызов техника-смотрителя, составление акта',
 'Грибок, плесень в квартире',
 'Опломбировать водосчетчик']

In [221]:
df_defect_kv = df_bez_vypolnenia[df_bez_vypolnenia['Наименование дефекта'].isin(def_adress_kv)][
    ['Наименование дефекта', 'Адрес проблемы', 'Квартира', 'Дата создания заявки в формате Timezone']
].sort_values(['Наименование дефекта', 'Адрес проблемы', 'Квартира', 'Дата создания заявки в формате Timezone'])
df_defect_kv['Следующий вызов'] = df_defect_kv.shift(-1)['Дата создания заявки в формате Timezone']
df_defect_kv['Следующий вызов'] = df_defect_kv['Следующий вызов'].where(
    (df_defect_kv['Наименование дефекта']==df_defect_kv['Наименование дефекта'].shift(-1))
    &(df_defect_kv['Адрес проблемы']==df_defect_kv['Адрес проблемы'].shift(-1))
    &(df_defect_kv['Квартира']==df_defect_kv['Квартира'].shift(-1)), np.nan)
df_defect_kv['Часов до повторного вызова'] = (df_defect_kv['Следующий вызов'] - df_defect_kv['Дата создания заявки в формате Timezone']) // np.timedelta64(1, 'h')
df_defect_kv

,Наименование дефекта,Адрес проблемы,Квартира,Дата создания заявки в формате Timezone,Следующий вызов,Часов до повторного вызова
1223439,"Грызуны (крысы, мыши) в квартире","1-й Тушинский проезд, дом 5",103,2021-03-04 10:45:22.742783+03:00,NaT,NaN
5556596,"Грызуны (крысы, мыши) в квартире","1-й Хорошёвский проезд, дом 10, корпус 1",12,2021-11-13 13:32:11.641643+03:00,NaT,NaN
4608695,"Грызуны (крысы, мыши) в квартире","1-й Хорошёвский проезд, дом 10, корпус 1",36,2021-09-23 15:40:04.137826+03:00,NaT,NaN
1478730,"Грызуны (крысы, мыши) в квартире","1-я Напрудная улица, дом 7",1,2021-03-18 20:22:28.938688+03:00,2021-03-24 18:18:38.735817+03:00,141.0
1572687,"Грызуны (крысы, мыши) в квартире","1-я Напрудная улица, дом 7",1,2021-03-24 18:18:38.735817+03:00,NaT,NaN
...,...,...,...,...,...,...
6469807,"Разрушение, трещины в стене, полу, потолке (кв...","13-я Парковая улица, дом 16А",60,2021-12-25 16:51:21.972875+03:00,NaT,NaN
3838369,"Разрушение, трещины в стене, полу, потолке (кв...","улица Обручева, дом 5",70,2021-08-22 15:24:19.486541+03:00,NaT,NaN
4962651,"Разрушение, трещины в стене, полу, потолке (кв...","Дмитровское шоссе, дом 55",60,2021-10-11 09:39:36.441941+03:00,NaT,NaN
5275267,"Разрушение, трещины в стене, полу, потолке (кв...","Туристская улица, дом 14",104,2021-10-27 13:27:53.889758+03:00,2021-11-14 13:32:50.340619+03:00,432.0


In [222]:
df_defect_kv = df_defect_kv.dropna(subset=['Часов до повторного вызова'])[['Наименование дефекта', 'Адрес проблемы', 'Квартира', 'Часов до повторного вызова']]
df_defect_kv

,Наименование дефекта,Адрес проблемы,Квартира,Часов до повторного вызова
1478730,"Грызуны (крысы, мыши) в квартире","1-я Напрудная улица, дом 7",1,141.0
2786502,"Грызуны (крысы, мыши) в квартире","Абрамцевская улица, дом 24, корпус 1",93,1405.0
6467194,"Грызуны (крысы, мыши) в квартире","Алтуфьевское шоссе, дом 93",183,49.0
3978102,"Грызуны (крысы, мыши) в квартире","Багратионовский проезд, дом 6, корпус 2",31,4.0
1947551,"Грызуны (крысы, мыши) в квартире","Бибиревская улица, дом 17",77,243.0
...,...,...,...,...
6446440,"Разрушение, трещины в стене, полу, потолке (кв...","улица Юных Ленинцев, дом 105, корпус 1",61,72.0
4196341,"Разрушение, трещины в стене, полу, потолке (кв...","проспект Маршала Жукова, дом 48",75,133.0
5622471,"Разрушение, трещины в стене, полу, потолке (кв...","улица Константина Федина, дом 13",145,33.0
5658011,"Разрушение, трещины в стене, полу, потолке (кв...","улица Константина Федина, дом 13",145,5.0


In [223]:
dict_sroki = {x[0]: 0 if x[2]=='-' else int(x[2]) for x in df_pril3_sroki.itertuples()}
dict_sroki

{'Аварийный провал грунта, асфальта': 24,
 'Благоустройство дворовой территории после проведенных ремонтных, строительных работ': 72,
 'Благоустройство Другое': 0,
 'Восстановление бордюрного камня (дворовая территория)': 72,
 'Грязь, мусор на контейнерной площадке (дворовая территория)': 12,
 'Грязь, мусор, переполненные урны (дворовая территория)': 6,
 'Завезти, заменить песок в песочнице': 72,
 'Заказ бункера для вывоза строительного, крупногабаритного мусора': 0,
 'Замена, установка домовых знаков (указателей улицы, дома, подъезда)': 72,
 'Засор ливневой канализации, подтопление (дворовая территория)': 24,
 'Засохшие деревья, кустарники (сухостой)': 72,
 'Наклонившееся дерево, ветка': 72,
 'Несанкционированные объявления, надписи, рисунки (фасад, дворовая территория)': 24,
 'Открыть/закрыть вход на площадку (спортивную, для выгула собак)': 0,
 'Переполненные мусорные контейнеры (дворовая территория)': 12,
 'Просел, не зафиксирован люк, водоприемная решетка (дворовая территория)': 2

In [226]:
df_defect_kv['Наименование дефекта'].apply(lambda x: 0 if x not in dict_sroki else dict_sroki[x])

1478730     72
2786502     72
6467194     72
3978102     72
1947551     72
          ... 
6446440    168
4196341    168
5622471    168
5658011    168
5275267    168
Name: Наименование дефекта, Length: 119129, dtype: int64

In [227]:
df_defect_kv['Норматив'] = df_defect_kv['Наименование дефекта'].apply(lambda x: 0 if x not in dict_sroki else dict_sroki[x])
df_defect_kv

,Наименование дефекта,Адрес проблемы,Квартира,Часов до повторного вызова,Норматив
1478730,"Грызуны (крысы, мыши) в квартире","1-я Напрудная улица, дом 7",1,141.0,72
2786502,"Грызуны (крысы, мыши) в квартире","Абрамцевская улица, дом 24, корпус 1",93,1405.0,72
6467194,"Грызуны (крысы, мыши) в квартире","Алтуфьевское шоссе, дом 93",183,49.0,72
3978102,"Грызуны (крысы, мыши) в квартире","Багратионовский проезд, дом 6, корпус 2",31,4.0,72
1947551,"Грызуны (крысы, мыши) в квартире","Бибиревская улица, дом 17",77,243.0,72
...,...,...,...,...,...
6446440,"Разрушение, трещины в стене, полу, потолке (кв...","улица Юных Ленинцев, дом 105, корпус 1",61,72.0,168
4196341,"Разрушение, трещины в стене, полу, потолке (кв...","проспект Маршала Жукова, дом 48",75,133.0,168
5622471,"Разрушение, трещины в стене, полу, потолке (кв...","улица Константина Федина, дом 13",145,33.0,168
5658011,"Разрушение, трещины в стене, полу, потолке (кв...","улица Константина Федина, дом 13",145,5.0,168


In [245]:
df_defect_v1_kv = df_defect_kv[df_defect_kv['Часов до повторного вызова']<df_defect_kv['Норматив']]
df_defect_v1_kv

,Наименование дефекта,Адрес проблемы,Квартира,Часов до повторного вызова,Норматив
6467194,"Грызуны (крысы, мыши) в квартире","Алтуфьевское шоссе, дом 93",183,49.0,72
3978102,"Грызуны (крысы, мыши) в квартире","Багратионовский проезд, дом 6, корпус 2",31,4.0,72
6310389,"Грызуны (крысы, мыши) в квартире","Давыдковская улица, дом 10, корпус 6",105,19.0,72
2195731,"Грызуны (крысы, мыши) в квартире","Дмитровское шоссе, дом 86",122,1.0,72
3727304,"Грызуны (крысы, мыши) в квартире","Затонная улица, дом 5, корпус 4",21,42.0,72
...,...,...,...,...,...
4822743,"Разрушение, трещины в стене, полу, потолке (кв...","проспект Мира, дом 29",11,47.0,168
6446440,"Разрушение, трещины в стене, полу, потолке (кв...","улица Юных Ленинцев, дом 105, корпус 1",61,72.0,168
4196341,"Разрушение, трещины в стене, полу, потолке (кв...","проспект Маршала Жукова, дом 48",75,133.0,168
5622471,"Разрушение, трещины в стене, полу, потолке (кв...","улица Константина Федина, дом 13",145,33.0,168


In [249]:
# Top20 аномалий в датасете по шаблону 1 'Заявки, закрытые без выполнения. Повторные'
# Квартиры
df_defect_v1_kv.value_counts('Наименование дефекта')[:20]

Наименование дефекта
Отсутствие отопления в комнате, квартире                                                                                        11851
Непрогрев приборов отопления в комнате, квартире                                                                                 4302
Отсутствует электричество во всей квартире                                                                                       4008
Вызов техника-смотрителя, составление акта                                                                                       3962
Сильная течь по потолку, стенам с верхнего этажа (квартира)                                                                      3470
Слабая течь по потолку, стенам с верхнего этажа (квартира)                                                                       3079
Отсутствует горячая вода (ГВС)                                                                                                   2965
Слабая течь стояка водоснабжения в технич

In [237]:
df.loc[df_defect_v1.index]

,Корневой ИД заявки,ИД версии заявки,Номер заявки,Уникальный номер обращения ГУ (mos.ru),Дата создания заявки в формате Timezone,Дата начала действия версии заявки в формате Timezone,Наименование источника поступления заявки,Код источника поступления заявки,Имя создателя заявки,Признак инцидента,...,Дата закрытия,Желаемое время с,Желаемое время до,Дата отзыва/оценки,Отзыв,Оценка качества выполнения работ,Код оценки качества выполнения ра,Наименование категории платности,Код категории платности,Признак оплаты картой
6467194,106193182,106387164,06470407/21,NaN,2021-12-25 15:10:54.173380+03:00,2021-12-27 13:54:02.142282+03:00,телефон,phone,frontline,Нет,...,2021-12-27 13:54:02.142282+03:00,2021-12-25 16:00:00+03:00,2021-12-25 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
3978102,93839045,93839641,03981226/21,NaN,2021-08-31 08:59:17.928396+03:00,2021-08-31 09:05:22.758503+03:00,телефон,phone,frontline,Нет,...,2021-08-31 09:05:22.758503+03:00,2021-08-31 09:00:00+03:00,2021-08-31 14:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
6310389,105419968,105421743,06313572/21,0002-9000005-207306-0911653/21,2021-12-19 14:04:05.599346+03:00,2021-12-19 14:18:25.864326+03:00,МП Госуслуги,mobile,emp2,Нет,...,2021-12-19 14:18:25.864326+03:00,2021-12-20 06:00:00+03:00,2021-12-20 12:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
2195731,84201625,84207220,02197540/21,0002-9000005-207306-0270277/21,2021-05-03 14:47:56.594351+03:00,2021-05-03 16:17:54.718514+03:00,МП Госуслуги,mobile,emp2,Нет,...,2021-05-03 16:17:54.718514+03:00,2021-05-03 14:00:00+03:00,2021-05-03 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
3727304,92519600,92520371,03730410/21,NaN,2021-08-15 14:15:39.356242+03:00,2021-08-15 14:28:45.538033+03:00,телефон,phone,frontline,Нет,...,2021-08-15 14:28:45.538033+03:00,2021-08-15 15:00:00+03:00,2021-08-15 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4822743,97936434,97937042,04825885/21,NaN,2021-10-03 14:16:19.029216+03:00,2021-10-03 14:24:37.206186+03:00,телефон,phone,frontline,Нет,...,2021-10-03 14:24:37.206186+03:00,2021-10-03 15:00:00+03:00,2021-10-03 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
6446440,106092363,106123047,06449630/21,NaN,2021-12-24 16:15:17.441769+03:00,2021-12-24 20:21:12.856319+03:00,телефон,phone,frontline,Нет,...,2021-12-24 20:21:12.856319+03:00,2021-12-24 17:00:00+03:00,2021-12-24 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
4196341,94973757,95035865,04199466/21,NaN,2021-09-11 22:57:52.742430+03:00,2021-09-13 07:59:11.941390+03:00,Электронный дом,elec_house,ElecH,Нет,...,2021-09-13 07:59:11.941390+03:00,2021-09-12 11:00:00+03:00,2021-09-12 20:00:00+03:00,2021-09-14 18:26:44.350336+03:00,NaN,Плохо,bad,бесплатная,free,Нет
5622471,101965021,101966636,05625652/21,NaN,2021-11-17 08:21:55.548379+03:00,2021-11-17 08:42:41.004113+03:00,Электронный дом,elec_house,ElecH,Нет,...,2021-11-17 08:42:41.004113+03:00,2021-11-17 09:00:00+03:00,2021-11-17 18:00:00+03:00,2021-11-17 17:30:53.667474+03:00,NaN,Плохо,bad,бесплатная,free,Нет


In [239]:
# Выберем категории дефектов, где считаем аномалией повторное обращение по адресу и подъезду
def_adress_pod = df_pril3_sroki[df_pril3_sroki['Повторное локация']=='адрес+подъезд'].index.to_list()
# Первые 10 записей
def_adress_pod[:10]

['Неисправность системы вентиляции в подъезде',
 'Неисправен доводчик, магнит входной двери с домофоном',
 'Неисправен домофон, кодовый замок входной двери подъезда',
 'Течь в кабине, шахте лифта',
 'Хранение химических веществ, реагентов, строительных материалов в подъезде, во дворе',
 'Запах канализации в подъезде (засор, течь лежака/стояка)',
 'Повреждение, протечка козырька подъезда',
 'Сильная течь по потолку, стенам в подъезде на последнем этаже (протечка крыши)',
 'Слабая течь по потолку, стенам в подъезде на последнем этаже (протечка крыши)',
 'Течь на балкон с крыши (на последнем этаже)']

In [242]:
df_defect_pod = df_bez_vypolnenia[df_bez_vypolnenia['Наименование дефекта'].isin(def_adress_pod)][
    ['Наименование дефекта', 'Адрес проблемы', 'Подъезд', 'Дата создания заявки в формате Timezone']
].sort_values(['Наименование дефекта', 'Адрес проблемы', 'Подъезд', 'Дата создания заявки в формате Timezone'])
df_defect_pod['Следующий вызов'] = df_defect_pod.shift(-1)['Дата создания заявки в формате Timezone']
df_defect_pod['Следующий вызов'] = df_defect_pod['Следующий вызов'].where(
    (df_defect_pod['Наименование дефекта']==df_defect_pod['Наименование дефекта'].shift(-1))
    &(df_defect_pod['Адрес проблемы']==df_defect_pod['Адрес проблемы'].shift(-1))
    &(df_defect_pod['Подъезд']==df_defect_pod['Подъезд'].shift(-1)), np.nan)
df_defect_pod['Часов до повторного вызова'] = (df_defect_pod['Следующий вызов'] - df_defect_pod['Дата создания заявки в формате Timezone']) // np.timedelta64(1, 'h')
df_defect_pod

,Наименование дефекта,Адрес проблемы,Подъезд,Дата создания заявки в формате Timezone,Следующий вызов,Часов до повторного вызова
1558065,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 4",12,2021-03-23 21:00:41.984148+03:00,2021-03-29 16:07:33.808503+03:00,139.0
1649215,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 4",12,2021-03-29 16:07:33.808503+03:00,NaT,NaN
6383442,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 7, корпус 1",1,2021-12-22 12:33:11.397789+03:00,2021-12-22 12:35:12.487604+03:00,0.0
6383526,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 7, корпус 1",1,2021-12-22 12:35:12.487604+03:00,NaT,NaN
5931030,"Грибок, плесень в подъезде","1-й Тушинский проезд, дом 7",3,2021-12-02 17:28:35.174187+03:00,NaT,NaN
...,...,...,...,...,...,...
6142346,"Течь в кабине, шахте лифта","Элеваторная улица, дом 6, корпус 1",1,2021-12-12 15:15:44.212224+03:00,NaT,NaN
6260875,"Течь в кабине, шахте лифта","улица Жуковского, дом 19, строение 1",1,2021-12-17 08:05:11.512369+03:00,NaT,NaN
4240041,"Течь в кабине, шахте лифта","Ярцевская улица, дом 31А",1,2021-09-14 13:07:55.303234+03:00,2021-09-14 16:17:58.100563+03:00,3.0
4245294,"Течь в кабине, шахте лифта","Ярцевская улица, дом 31А",1,2021-09-14 16:17:58.100563+03:00,NaT,NaN


In [243]:
df_defect_pod = df_defect_pod.dropna(subset=['Часов до повторного вызова'])[['Наименование дефекта', 'Адрес проблемы', 'Подъезд', 'Часов до повторного вызова']]
df_defect_pod

,Наименование дефекта,Адрес проблемы,Подъезд,Часов до повторного вызова
1558065,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 4",12,139.0
6383442,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 7, корпус 1",1,0.0
3253988,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,659.0
3655982,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,38.0
3678192,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,315.0
...,...,...,...,...
4308859,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,15.0
4321713,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,1.0
4330529,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,0.0
4330950,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,3.0


In [244]:
df_defect_pod['Норматив'] = df_defect_pod['Наименование дефекта'].apply(lambda x: 0 if x not in dict_sroki else dict_sroki[x])
df_defect_pod

,Наименование дефекта,Адрес проблемы,Подъезд,Часов до повторного вызова,Норматив
1558065,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 4",12,139.0,720
6383442,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 7, корпус 1",1,0.0,720
3253988,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,659.0,720
3655982,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,38.0,720
3678192,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,315.0,720
...,...,...,...,...,...
4308859,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,15.0,24
4321713,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,1.0,24
4330529,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,0.0,24
4330950,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,3.0,24


In [247]:
df_defect_v1_pod = df_defect_pod[df_defect_pod['Часов до повторного вызова']<df_defect_pod['Норматив']]
df_defect_v1_pod

,Наименование дефекта,Адрес проблемы,Подъезд,Часов до повторного вызова,Норматив
1558065,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 4",12,139.0,720
6383442,"Грибок, плесень в подъезде","1-й Саратовский проезд, дом 7, корпус 1",1,0.0,720
3253988,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,659.0,720
3655982,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,38.0,720
3678192,"Грибок, плесень в подъезде","15-я Парковая улица, дом 45",2,315.0,720
...,...,...,...,...,...
4308859,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,15.0,24
4321713,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,1.0,24
4330529,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,0.0,24
4330950,"Течь в кабине, шахте лифта","Старокоптевский переулок, дом 8А",1,3.0,24


In [248]:
# Top20 аномалий в датасете по шаблону 1 'Заявки, закрытые без выполнения. Повторные'
# Подъезды
df_defect_v1_pod.value_counts('Наименование дефекта')[:20]

Наименование дефекта
Неисправность осветительного прибора, замена лампы (подъезд)                                        6816
Все лифты или единственный лифт не работает (стоит, самопроизвольно движется, хлопает дверями)      4850
Неисправен домофон, кодовый замок входной двери подъезда                                            4466
Неисправен доводчик, магнит входной двери с домофоном                                               4462
Засор мусоропровода                                                                                 4096
Шум, гул, вибрация в системе отопления                                                              1869
Грязь, мусор в подъезде, на крыльце                                                                 1685
Один из лифтов не работает (стоит, самопроизвольно движется, хлопает дверями)                       1666
Сильная течь по потолку, стенам, лестнице с верхнего этажа (подъезд)                                1197
Перепады, скачки напряжения в элек

In [252]:
# Выберем категории дефектов, где считаем аномалией повторное обращение по адресу
def_adress = df_pril3_sroki[df_pril3_sroki['Повторное локация'].isin(['адрес', 'Адрес'])].index.to_list()
# Первые 10 записей
def_adress[:10]

['Аварийный провал грунта, асфальта',
 'Благоустройство дворовой территории после проведенных ремонтных, строительных работ',
 'Восстановление бордюрного камня (дворовая территория)',
 'Грязь, мусор на контейнерной площадке (дворовая территория)',
 'Грязь, мусор, переполненные урны (дворовая территория)',
 'Завезти, заменить песок в песочнице',
 'Замена, установка домовых знаков (указателей улицы, дома, подъезда)',
 'Засор ливневой канализации, подтопление (дворовая территория)',
 'Засохшие деревья, кустарники (сухостой)',
 'Наклонившееся дерево, ветка']

In [257]:
df_defect_addr = df_bez_vypolnenia[df_bez_vypolnenia['Наименование дефекта'].isin(def_adress)][
    ['Наименование дефекта', 'Адрес проблемы', 'Дата создания заявки в формате Timezone']
].sort_values(['Наименование дефекта', 'Адрес проблемы', 'Дата создания заявки в формате Timezone'])
df_defect_addr['Следующий вызов'] = df_defect_addr.shift(-1)['Дата создания заявки в формате Timezone']
df_defect_addr['Следующий вызов'] = df_defect_addr['Следующий вызов'].where(
    (df_defect_addr['Наименование дефекта']==df_defect_addr['Наименование дефекта'].shift(-1))
    &(df_defect_addr['Адрес проблемы']==df_defect_addr['Адрес проблемы'].shift(-1)), np.nan)
df_defect_addr['Часов до повторного вызова'] = (df_defect_addr['Следующий вызов'] - df_defect_addr['Дата создания заявки в формате Timezone']) // np.timedelta64(1, 'h')
df_defect_addr

,Наименование дефекта,Адрес проблемы,Дата создания заявки в формате Timezone,Следующий вызов,Часов до повторного вызова
2259666,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Аэропортовская улица, дом 6",2021-05-09 10:45:37.824920+03:00,NaT,NaN
1684367,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 21, корпус 1",2021-03-31 14:48:12.215993+03:00,2021-04-03 16:54:41.231484+03:00,74.0
1736458,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 21, корпус 1",2021-04-03 16:54:41.231484+03:00,2021-07-09 10:06:43.530347+03:00,2321.0
3188206,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 21, корпус 1",2021-07-09 10:06:43.530347+03:00,NaT,NaN
3572524,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 22, корпус 1",2021-08-05 08:31:29.044229+03:00,2021-08-16 09:23:24.004233+03:00,264.0
...,...,...,...,...,...
4778809,"Шум, гул в шахте лифта","Остафьевская улица, дом 8, корпус 2",2021-09-30 20:59:29.604804+03:00,NaT,NaN
5709243,"Шум, гул в шахте лифта","улица Алексея Дикого, дом 5, корпус 1",2021-11-21 17:28:28.874107+03:00,NaT,NaN
4562283,"Шум, гул в шахте лифта","улица Островитянова, дом 43, корпус 2",2021-09-22 12:40:16.436932+03:00,NaT,NaN
6224168,"Шум, гул в шахте лифта","Планерная улица, дом 3, корпус 3",2021-12-15 16:41:18.712985+03:00,NaT,NaN


In [258]:
df_defect_addr = df_defect_addr.dropna(subset=['Часов до повторного вызова'])[['Наименование дефекта', 'Адрес проблемы', 'Часов до повторного вызова']]
df_defect_addr

,Наименование дефекта,Адрес проблемы,Часов до повторного вызова
1684367,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 21, корпус 1",74.0
1736458,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 21, корпус 1",2321.0
3572524,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 22, корпус 1",264.0
37063,"Грызуны (крысы, мыши) в общедомовых помещениях","10-я улица Соколиной Горы, дом 20",3289.0
3256722,"Грызуны (крысы, мыши) в общедомовых помещениях","16-я Парковая улица, дом 18",0.0
...,...,...,...
4071046,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",1.0
4072245,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",2.0
4074991,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",43.0
4113028,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",0.0


In [262]:
df_defect_addr['Норматив'] = df_defect_addr['Наименование дефекта'].apply(lambda x: 0 if x not in dict_sroki else dict_sroki[x])
df_defect_addr

,Наименование дефекта,Адрес проблемы,Часов до повторного вызова,Норматив
1684367,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 21, корпус 1",74.0,72
1736458,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 21, корпус 1",2321.0,72
3572524,"Грызуны (крысы, мыши) в общедомовых помещениях","1-я Новокузьминская улица, дом 22, корпус 1",264.0,72
37063,"Грызуны (крысы, мыши) в общедомовых помещениях","10-я улица Соколиной Горы, дом 20",3289.0,72
3256722,"Грызуны (крысы, мыши) в общедомовых помещениях","16-я Парковая улица, дом 18",0.0,72
...,...,...,...,...
4071046,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",1.0,48
4072245,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",2.0,48
4074991,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",43.0,48
4113028,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",0.0,48


In [263]:
df_defect_v1_addr = df_defect_addr[df_defect_addr['Часов до повторного вызова']<df_defect_addr['Норматив']]
df_defect_v1_addr

,Наименование дефекта,Адрес проблемы,Часов до повторного вызова,Норматив
3256722,"Грызуны (крысы, мыши) в общедомовых помещениях","16-я Парковая улица, дом 18",0.0,72
4253186,"Грызуны (крысы, мыши) в общедомовых помещениях","2-й Крестовский переулок, дом 4",0.0,72
456784,"Грызуны (крысы, мыши) в общедомовых помещениях","9-я Парковая улица, дом 66, корпус 2",49.0,72
497558,"Грызуны (крысы, мыши) в общедомовых помещениях","9-я Парковая улица, дом 66, корпус 2",3.0,72
500662,"Грызуны (крысы, мыши) в общедомовых помещениях","9-я Парковая улица, дом 66, корпус 2",14.0,72
...,...,...,...,...
4071046,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",1.0,48
4072245,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",2.0,48
4074991,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",43.0,48
4113028,"Шум, гул в шахте лифта","проезд Серебрякова, дом 9, корпус 1",0.0,48


In [264]:
# Top20 аномалий в датасете по шаблону 1 'Заявки, закрытые без выполнения. Повторные'
# Адреса
df_defect_v1_addr.value_counts('Наименование дефекта')[:20]

Наименование дефекта
Насекомые в общедомовых помещениях                                                       6745
Затопление, пар в подвале, цокольном этаже, паркинге                                     2664
Шум, гул в подвале, на техническом этаже                                                 1842
Отсутствует электричество во всем подъезде, доме                                         1678
Неисправность элементов уличного освещения (дворовая территория)                          911
Некачественно произведенный капитальный ремонт (подъезд, квартира)                        492
Повреждение, засор наружного водостока                                                    343
Содержание, ремонт объектов благоустройства (МАФ) на дворовой территории                  302
Грызуны (крысы, мыши) в общедомовых помещениях                                            262
Наклонившееся дерево, ветка                                                               216
Животные на крыше, чердаке, в подвале д

In [266]:
df_defect_v1_kv.index

Int64Index([6467194, 3978102, 6310389, 2195731, 3727304, 2268729, 2296875,
            3569825, 1320696, 1781511,
            ...
            4991760, 5001573, 6334893, 6406808, 6447511, 4822743, 6446440,
            4196341, 5622471, 5658011],
           dtype='int64', length=70474)

In [267]:
# Суммарные аномалии по шаблону 1
index_template1 = list(df_defect_v1_kv.index) + list(df_defect_v1_pod.index) + list(df_defect_v1_addr.index)
len(index_template1)

133579

In [268]:
# Список заявок с аномалиями в датасете по шаблону 1 'Заявки, закрытые без выполнения. Повторные' (все типы локаций)
df.loc[index_template1]

,Корневой ИД заявки,ИД версии заявки,Номер заявки,Уникальный номер обращения ГУ (mos.ru),Дата создания заявки в формате Timezone,Дата начала действия версии заявки в формате Timezone,Наименование источника поступления заявки,Код источника поступления заявки,Имя создателя заявки,Признак инцидента,...,Дата закрытия,Желаемое время с,Желаемое время до,Дата отзыва/оценки,Отзыв,Оценка качества выполнения работ,Код оценки качества выполнения ра,Наименование категории платности,Код категории платности,Признак оплаты картой
6467194,106193182,106387164,06470407/21,NaN,2021-12-25 15:10:54.173380+03:00,2021-12-27 13:54:02.142282+03:00,телефон,phone,frontline,Нет,...,2021-12-27 13:54:02.142282+03:00,2021-12-25 16:00:00+03:00,2021-12-25 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
3978102,93839045,93839641,03981226/21,NaN,2021-08-31 08:59:17.928396+03:00,2021-08-31 09:05:22.758503+03:00,телефон,phone,frontline,Нет,...,2021-08-31 09:05:22.758503+03:00,2021-08-31 09:00:00+03:00,2021-08-31 14:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
6310389,105419968,105421743,06313572/21,0002-9000005-207306-0911653/21,2021-12-19 14:04:05.599346+03:00,2021-12-19 14:18:25.864326+03:00,МП Госуслуги,mobile,emp2,Нет,...,2021-12-19 14:18:25.864326+03:00,2021-12-20 06:00:00+03:00,2021-12-20 12:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
2195731,84201625,84207220,02197540/21,0002-9000005-207306-0270277/21,2021-05-03 14:47:56.594351+03:00,2021-05-03 16:17:54.718514+03:00,МП Госуслуги,mobile,emp2,Нет,...,2021-05-03 16:17:54.718514+03:00,2021-05-03 14:00:00+03:00,2021-05-03 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
3727304,92519600,92520371,03730410/21,NaN,2021-08-15 14:15:39.356242+03:00,2021-08-15 14:28:45.538033+03:00,телефон,phone,frontline,Нет,...,2021-08-15 14:28:45.538033+03:00,2021-08-15 15:00:00+03:00,2021-08-15 20:00:00+03:00,NaT,NaN,NaN,NaN,бесплатная,free,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4071046,94321521,94326726,04074170/21,NaN,2021-09-05 14:17:12.863957+03:00,2021-09-05 15:23:47.964751+03:00,телефон,phone,voxcom,Нет,...,2021-09-05 15:23:47.964751+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
4072245,94327196,94340052,04075369/21,NaN,2021-09-05 15:29:59.513757+03:00,2021-09-05 18:19:06.969856+03:00,телефон,phone,frontline,Нет,...,2021-09-05 18:19:06.969856+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
4074991,94340541,94346874,04078115/21,NaN,2021-09-05 18:25:44.990224+03:00,2021-09-05 20:07:51.752166+03:00,телефон,phone,frontline,Нет,...,2021-09-05 20:07:51.752166+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет
4113028,94530559,94584031,04116153/21,NaN,2021-09-07 13:42:01.452953+03:00,2021-09-07 20:56:11.676850+03:00,телефон,phone,frontline,Нет,...,2021-09-07 20:56:11.676850+03:00,NaT,NaT,NaT,NaN,NaN,NaN,бесплатная,free,Нет


In [270]:
df.loc[index_template1].to_pickle('../data/anomalii_1.pickle')